### Sampling the outputs to send for annotations 
This is for research/development purposes. It extracts random sample from Step_A's output. The samples were sent to manual annotators. The manual annotations are used in Step_C onward. Step_B does not use this notebook's output.

In [ ]:
from cw_package.setup_cw_env import *
from pylab import *
from cw_package import prDF
from au_package import assess_agreement, prep_Au_standard, standardize_flags
import pickle
#import re
jacks_verification

In [ ]:
coded_date = '2017_05_14'
earliest_date= '2016_04_01'
time_period= ['2016_4','2016_5','2016_6','2016_7','2016_8','2016_9']
elevenkeys= ['CW_cerv','CW_card','CW_vitd','CW_bph','CW_lbp','CW_feed','CW_psyc','CW_dexa','CW_narc',
             'CW_nonpreop','CW_catpreop']
stdflag_string= 'as_annotated'

GEMdicts = ['10to10', 'claimto9best', 'claimto9reimb', 'refto10best','refto10gems']

### Import preprocessed data

In [ ]:
# Load compiled numerators and denominators from different GEM implementations
numerators_lt   = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_compilednumerators_'+coded_date+'_long.p')
denominators_lt = pd.read_pickle('./preprocessed/'+coded_date+'/pickled_compileddenominators_'+coded_date+'_long.p')

In [ ]:
gem = 1
met = 'CW_bph'
print(numerators_lt[gem][0])
a = numerators_lt[gem][1][numerators_lt[gem][1].Metric==met]
b= denominators_lt[gem][1][denominators_lt[gem][1].Metric==met]

In [ ]:
mrn = '<insert_mrn_here>'
a[a.MRN==mrn]

#print(b[b.MRN==mrn])

Removing the numerators from the denominators in each GEM implementation to ensure no repeated sampling

In [ ]:
sel_numerators_lt = [None]*len(numerators_lt)
for x in range(len(numerators_lt)):
    sel_numerators_lt[x]=(numerators_lt[x][0],
                          numerators_lt[x][1][numerators_lt[x][1].TEST_DATE_month.isin(time_period)])

In [ ]:
notnumerators_lt = [None]*len(numerators_lt)
for x in range(len(numerators_lt)):
    if denominators_lt[x][0][3:]!=sel_numerators_lt[x][0][3:]:
        raise ValueError('numerators and denominators have different GEMs or order to GEMs')
    else:
        tempdf=(denominators_lt[x][0][3:], 
                        pd.concat([denominators_lt[x][1], numerators_lt[x][1]]).drop_duplicates(
                        subset=['Metric','MRN','TEST_DATE_a','TEST_CODE'], keep=False))
        notnumerators_lt[x]=(tempdf[0],tempdf[1][tempdf[1].TEST_DATE_month.isin(time_period)])
    

In [ ]:
#notnumerators_lt[1]

At this point,
numerators_lt and notnumerators_lt are the analyses' results for assigning numerator status
(rather than numerator and denominator)

In [ ]:
for x in notnumerators_lt:
    x[1].loc[:,'Term per GEM eval']='Not_Numer'
for x in sel_numerators_lt:
    x[1].loc[:,'Term per GEM eval']='Numer'
    
# concatenate them into single dataframes by Metric, w/in individual GEM implementations
allterms_lt = [None]*len(numerators_lt)
for x in range(len(numerators_lt)):
    allterms_lt[x]=(sel_numerators_lt[x][0][3:], pd.concat([notnumerators_lt[x][1], sel_numerators_lt[x][1]]))
    allterms_lt[x][1][allterms_lt[x][0]]=1

In [ ]:
for x in allterms_lt:
    print(x[0])

In [ ]:
refer = pd.read_csv('../selfreview/newsampling/metricsneeded_'+coded_date+'.csv')

### For Performing Samples to Send for Annotation

In [ ]:
listd=list(denominators_lt[1][1].columns.values)
listn=list(numerators_lt[1][1].columns.values)

In [ ]:
# dflist[0][1].sort_values('TEST_DATE_a').columns.values
listofcolumns10 = ['ADM_DT', 'ADM_DT_a', 
       'AgeAtTest', 'CLAIM_DATE', 'CLAIM_DATE_culled',
       'DISP_AMT', 'DISP_UNIT', 'DIS_DT', 'DOB', 'DOSE_FORM',
       'Drug Names', 'FIRST_NAME', 'FREQUENCY',
       'GENDER', 'ICD10_subcode', 'ICD10sub_culled', 'IDX_DATE_a',
       'MED_NAME', 'MRN', 'MRN_culled', 'Metric', 
       'RX_DOSE', 'RX_UNIT', 'SURG_CODE', 'SURG_DATE', 'SURG_ENC_ID',
       'TEST_CODE', 'TEST_CODE_culled', 'TEST_DATE_a', 
       'TEST_DATE_culled', 'ccslev', 'ccslev_culled',
       'daysBeforeSurg', 'dys_difftime_lastdischarge',
       'dys_difftime_noadms_criteria', 'dys_difftime_test_code', 'hcclev',
       'hcclev_culled']

listofcolumns9= ['ADM_DT', 'ADM_DT_a', 
       'AgeAtTest', 'CLAIM_DATE', 
       'DISP_AMT', 'DISP_UNIT', 'DIS_DT', 'DOB',
       'Drug Names', 'FIRST_NAME', 'FREQUENCY',
       'GENDER', 'ICD9_subcode',
                 #'ICD9sub_culled',
       'IDX_DATE_a',
       'MED_NAME', 'MRN', 
                 #'MRN_culled', 
       'Metric', 
       'RX_DOSE',  'SURG_CODE', 'SURG_DATE', 
       'TEST_CODE', 
                 #'TEST_CODE_culled', 
       'TEST_DATE_a',
                 #'TEST_DATE_culled', 
       'ccslev', 
                 #'ccslev_culled',
       'daysBeforeSurg', 'dys_difftime_lastdischarge',
       'dys_difftime_noadms_criteria', 'dys_difftime_test_code', 'hcclev',
       #'hcclev_culled', 
       'rCODE','rMRN']

prioritycolumns9  = ['Metric', 'MRN', 'AgeAtTest', 'CLAIM_DATE', 'ICD9_subcode', 'TEST_CODE','Drug Names','DOSE_FORM','TEST_DATE_a', 'hcclev','ccslev', 'ADM_DT','SURG_DATE','dys_difftime_test_code','daysBeforeSurg','dys_difftime_noadms_criteria','dys_difftime_lastdischarge']
prioritycolumns10 = ['Metric', 'MRN', 'AgeAtTest', 'CLAIM_DATE', 'ICD10_subcode','TEST_CODE','Drug Names','DOSE_FORM','TEST_DATE_a','hcclev','ccslev', 'ADM_DT','SURG_DATE','dys_difftime_test_code','daysBeforeSurg','dys_difftime_noadms_criteria','dys_difftime_lastdischarge']


In [ ]:
bean =0
for x in listd:
    if x not in listofcolumns9:
        print(x)
    else:
        bean+=1
print(bean)

In [ ]:
# declare local arguments
dflist=[]
toofew = []
GEMindex = 1

priorityused = prioritycolumns9
listXused    = listofcolumns9
finalcolumns = priorityused + [x for x in listXused if x not in priorityused]

In [ ]:
tenkeys= ['CW_cerv','CW_card','CW_vitd','CW_bph','CW_lbp','CW_psyc','CW_dexa','CW_narc',
             'CW_nonpreop','CW_catpreop']

In [ ]:
# for column name testing --- used for development of sampling method 

"""
for x in elevenkeys:
    print(x)
    try:
        if x=='CW_feed':
            pass # feeding metrics sampled separately due to question of whether ICD procedural codes used
        else:
            hold_N= sel_numerators_lt[GEMindex][1][sel_numerators_lt[GEMindex][1].Metric==x]
            number_N=refer.loc[refer.Metric==x,'Numer needed'].iloc[0]
            print('number_N: {}'.format(number_N))

            if number_N>0:
                if len(hold_N)<number_N:
                    pass
                else:
                    # randomly select cases for review
                    hold_N=hold_N.sample(n=number_N, random_state=54)

                print('sel_numerators GEM {}, {}'.format(sel_numerators_lt[GEMindex][0],x))
                print('notnumerators GEM {}, {}'.format(notnumerators_lt[GEMindex][0],x))
            else:
                pass
            hold_NN= notnumerators_lt[GEMindex][1][notnumerators_lt[GEMindex][1].Metric==x]
            number_NN=refer.loc[refer.Metric==x,'Not_Numer needed'].iloc[0]
            print('number_NN: {}'.format(number_NN))
            if number_NN>0:
                if len(hold_NN)<number_NN:
                    pass
                else: 
                    # randomly select cases for review
                    hold_NN=hold_NN.sample(n=number_NN, random_state=54)
            else:
                pass
            if (number_N+number_NN)>0:
                # blind the assignment (numerator/not-numerator) prior to exporting table
                outdf = pd.concat([hold_N.drop('Term per GEM eval', axis=1), hold_NN.drop('Term per GEM eval', axis=1)])
                try:
                    outdf = outdf[finalcolumns].dropna(axis=1, how='all')
                    print(outdf.head(2))
                except:
                    for x in finalcolumns:
                        if x not in hold_N:
                            print(x)
                        elif x not in hold_NN:
                            print('NN')
                            print(xx)
                    raise ValueError('Forced columns are incorrect')
                dflist.append((x,outdf.sort_values('TEST_DATE_a')))

                if len(hold_N)<number_N or len(hold_NN)<number_NN:
                    toofew.append(x)
                print('success x1')
            else:
                print('none required')
                pass
    except:
        print('{} is passed'.format(x))
"""

In [ ]:

for x in elevenkeys:
    try:
        hold_N= sel_numerators_lt[GEMindex][1][sel_numerators_lt[GEMindex][1].Metric==x]
        number_N=refer.loc[refer.Metric==x,'Numer needed'].iloc[0]
        if len(hold_N)<number_N:
            pass
        else:
            # randomly select cases for iterative review
            hold_N=hold_N.sample(n=number_N, random_state=54)

        print('sel_numerators GEM {}, {}'.format(sel_numerators_lt[GEMindex][0],x))
        print('notnumerators GEM {}, {}'.format(notnumerators_lt[GEMindex][0],x))
        hold_NN= notnumerators_lt[GEMindex][1][notnumerators_lt[GEMindex][1].Metric==x]
        number_NN=refer.loc[refer.Metric==x,'Not_Numer needed'].iloc[0]
        if len(hold_NN)<number_NN:
            pass
        else: 
            # randomly select cases for iterative review
            hold_NN=hold_NN.sample(n=number_NN, random_state=54)
        # blind the assignment prior to exporting cases for iterative review
        outdf = pd.concat([hold_N.drop('Term per GEM eval', axis=1), hold_NN.drop('Term per GEM eval', axis=1)])
        try:
            outdf = outdf[finalcolumns].dropna(axis=1, how='all')
        except:
            for x in finalcolumns:
                if x not in hold_N:
                    print(x)
                elif x not in hold_NN:
                    print('NN')
                    print(xx)
            raise ValueError('Forced columns are incorrect')
        dflist.append((x,outdf.sort_values('TEST_DATE_a')))

        if len(hold_N)<number_N or len(hold_NN)<number_NN:
            toofew.append(x)
        print('success x1')
    except:
        print('{} is passed'.format(x))


Now perform special accommodation for feeding tube metric

In [ ]:
# declare local arguments
dffeed=[]
toofew_f = []
# choice of GEM for case review
GEMindex = 4

priorityused = prioritycolumns10
listXused    = listofcolumns10
finalcolumns = priorityused + [x for x in listXused if x not in priorityused]

In [ ]:
# this was used to check that no ICD procedural codes were used in patient data
for x in ['CW_feed']:
    try:
        hold_N= sel_numerators_lt[GEMindex][1][sel_numerators_lt[GEMindex][1].Metric==x]
        number_N=refer.loc[refer.Metric==x,'Numer needed'].iloc[0]
        if len(hold_N)<number_N:
            pass
        else:
            hold_N=hold_N.sample(n=number_N, random_state=54)
        
        print('sel_numerators GEM {}, {}'.format(sel_numerators_lt[GEMindex][0],x))
        print('notnumerators GEM {}, {}'.format(notnumerators_lt[GEMindex][0],x))
        hold_NN= notnumerators_lt[GEMindex][1][notnumerators_lt[GEMindex][1].Metric==x]
        number_NN=refer.loc[refer.Metric==x,'Not_Numer needed'].iloc[0]
        if len(hold_NN)<number_NN:
            pass
        else: 
            hold_NN=hold_NN.sample(n=number_NN, random_state=54)
        outdf = pd.concat([hold_N.drop('Term per GEM eval', axis=1), hold_NN.drop('Term per GEM eval', axis=1)])
        try:
            outdf = outdf[finalcolumns].dropna(axis=1, how='all')
        except:
            raise ValueError('Forced columns are incorrect')
        dffeed.append((x,outdf.sort_values('TEST_DATE_a')))
        
        if len(hold_N)<number_N or len(hold_NN)<number_NN:
            toofew_f.append(x)
    except:
        print('{} is passed'.format(x))

In [ ]:
# export the cases to file to share with reviewers
filex = pd.ExcelWriter('../selfreview/newsampling/'+coded_date+'_newsample.xlsx', engine='xlsxwriter')
for x in dflist:
    x[1].to_excel(filex, x[0])
for x in dffeed:
    x[1].to_excel(filex, x[0])
filex.save()

In [ ]:
avail_lt = pd.concat([x[1].groupby(['Metric','Term per GEM eval']).count() for x in allterms_lt], axis=1)
avail_lt = avail_lt[['_'+x for x in GEMdicts]]
avail_lt